The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install d2l==1.0.3


# Custom Layers

One factor behind deep learning's success
is the availability of a wide range of layers
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.
For instance, researchers have invented layers
specifically for handling images, text,
looping over sequential data,
and
performing dynamic programming.
Sooner or later, you will need
a layer that does not exist yet in the deep learning framework.
In these cases, you must build a custom layer.
In this section, we show you how.


In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## (**Layers without Parameters**)

To start, we construct a custom layer
that does not have any parameters of its own.
This should look familiar if you recall our
introduction to modules in :numref:`sec_model_construction`.
The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [ ]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

Let's verify that our layer works as intended by feeding some data through it.


In [ ]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can now [**incorporate our layer as a component
in constructing more complex models.**]


In [ ]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

As an extra sanity check, we can send random data
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers,
we may still see a very small nonzero number
due to quantization.


In [ ]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-6.5193e-09, grad_fn=<MeanBackward0>)

## [**Layers with Parameters**]

Now that we know how to define simple layers,
let's move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let's implement our own version of the  fully connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires two input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [ ]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

Next, we instantiate the `MyLinear` class
and access its model parameters.


In [ ]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.4783,  0.4284, -0.0899],
        [-0.6347,  0.2913, -0.0822],
        [-0.4325, -0.1645, -0.3274],
        [ 1.1898,  0.6482, -1.2384],
        [-0.1479,  0.0264, -0.9597]], requires_grad=True)

We can [**directly carry out forward propagation calculations using custom layers.**]


In [ ]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.9316, 0.0000],
        [0.1808, 1.4208, 0.0000]])

We can also (**construct models using custom layers.**)
Once we have that we can use it just like the built-in fully connected layer.


In [ ]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 0.0000],
        [13.0800]])

## Summary

We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
Once defined, custom layers can be invoked in arbitrary contexts and architectures.
Layers can have local parameters, which can be created through built-in functions.


## Exercises

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.


[Discussions](https://discuss.d2l.ai/t/59)


### Exercise 1:

In [19]:
class TensorReduction(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(units, in_units, in_units))

    def forward(self, X):
        X_reshaped = X.unsqueeze(1)
        return torch.einsum('bni,uij,bnj->bu', X_reshaped, self.weights, X_reshaped)

In [20]:
layer = TensorReduction(5, 2)
layer.weights

Parameter containing:
tensor([[[ 0.4558,  0.6859,  1.4242,  0.4022, -0.2911],
         [ 0.9249,  1.3740,  0.8691, -0.3274, -2.3215],
         [ 1.0581,  0.2569, -0.6168,  0.1010,  1.3597],
         [ 0.0228,  0.1745, -0.3105,  0.0185,  1.8650],
         [-1.6680, -0.0581,  0.7627,  0.6545, -0.0721]],

        [[ 0.0738,  0.3573, -0.6933, -1.0141, -0.2564],
         [-0.5453, -0.3533,  0.6772, -0.0432, -0.2741],
         [-0.0964, -1.4810,  0.5322,  0.1129, -0.3879],
         [-0.9456,  0.6494,  0.2096,  0.8750,  0.7875],
         [ 1.8252,  0.4363,  0.0748, -0.5183, -0.6972]]], requires_grad=True)

In [21]:
layer(torch.randn(2, 5))

tensor([[8.5542, 0.3068],
        [2.1325, 6.0718]], grad_fn=<ViewBackward0>)

### Exercise 2:

In [43]:
import torch
import torch.nn as nn

class FourierLayer(nn.Module):
    def __init__(self, keep_real=True):
        """
        A custom PyTorch layer that returns the leading half of the Fourier coefficients.

        Args:
            keep_real (bool): If True, keeps only the real part of the coefficients.
                              If False, returns both real and imaginary parts.
        """
        super().__init__()
        self.keep_real = keep_real  # Option to keep only real part

    def forward(self, X):
        """
        Compute the Fourier Transform and return the leading half of the coefficients.

        Args:
            X (torch.Tensor): Input tensor of shape (batch_size, sequence_length).

        Returns:
            torch.Tensor: The leading half of the Fourier coefficients.
        """
        # Compute the 1D FFT along the last dimension
        fft_coeffs = torch.fft.fft(X, dim=-1)  # Shape: (batch_size, sequence_length)

        # Keep only the first half of the coefficients
        half_size = X.shape[-1] // 2
        fft_coeffs = fft_coeffs[:, :half_size]

        # Optionally keep only the real part
        if self.keep_real:
            fft_coeffs = fft_coeffs.real  # Shape: (batch_size, half_size)

        return fft_coeffs


In [44]:
fourier = FourierLayer()

In [45]:
fourier(torch.randn(2,3))

tensor([[ 1.1488],
        [-0.8196]])